In [1]:
from dowhy import CausalModel
import dowhy.datasets

In [2]:
# Load some sample data
data = dowhy.datasets.linear_dataset(
    beta=10,
    num_common_causes=5,
    num_instruments=2,
    num_samples=10000,
    treatment_is_binary=True)

In [6]:
data['df']

,Z0,Z1,W0,W1,W2,W3,W4,v0,y
0,1.0,0.257419,-0.589486,-0.191323,-0.155594,0.781986,0.905694,True,13.223942
1,0.0,0.945265,0.409616,-0.253877,-1.507007,0.394412,-0.519735,True,4.357936
2,1.0,0.972292,-0.966976,0.700045,-0.688404,-0.995121,0.087379,True,5.879249
3,1.0,0.065989,0.645979,-0.914899,-1.193067,0.509132,0.118110,True,7.548974
4,0.0,0.896323,0.764813,-1.618964,1.746237,0.460453,-0.309666,True,13.205785
...,...,...,...,...,...,...,...,...,...
9995,1.0,0.360614,-1.437514,0.309709,0.377473,1.439038,0.375351,True,12.779996
9996,1.0,0.501683,-1.306996,0.284648,-0.631595,-0.791722,2.369436,True,14.849244
9997,0.0,0.606275,-0.398009,-1.537362,-1.336713,1.026779,0.895350,True,8.055161
9998,0.0,0.585278,-0.234223,-1.468885,0.706999,1.314633,1.171854,True,16.228463


In [7]:
# I. Create a causal model from the data and given graph.
model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])

# II. Identify causal effect and return target estimands
identified_estimand = model.identify_effect()

# III. Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")

# IV. Refute the obtained estimate using multiple robustness checks.
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

propensity_score_matching


/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A 

/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A 

/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A 

/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/martin/miniconda3/envs/causality/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A 

In [26]:
# dir(identified_estimand)
identified_estimand.estimands

{'backdoor1': {'estimand': Derivative(Expectation(y|W4,Z1,W1,Z0,W2,W3,W0), [v0]),
  'assumptions': {'Unconfoundedness': 'If U→{v0} and U→y then P(y|v0,W4,Z1,W1,Z0,W2,W3,W0,U) = P(y|v0,W4,Z1,W1,Z0,W2,W3,W0)'}},
 'backdoor2': {'estimand': Derivative(Expectation(y|W4,W1,W2,W3,W0), [v0]),
  'assumptions': {'Unconfoundedness': 'If U→{v0} and U→y then P(y|v0,W4,W1,W2,W3,W0,U) = P(y|v0,W4,W1,W2,W3,W0)'}},
 'backdoor': {'estimand': Derivative(Expectation(y|W4,W1,W2,W3,W0), [v0]),
  'assumptions': {'Unconfoundedness': 'If U→{v0} and U→y then P(y|v0,W4,W1,W2,W3,W0,U) = P(y|v0,W4,W1,W2,W3,W0)'}},
 'iv': {'estimand': Expectation(Derivative(y, [Z1, Z0])*Derivative([v0], [Z1, Z0])**(-1)),
  'assumptions': {'As-if-random': 'If U→→y then ¬(U →→{Z1,Z0})',
   'Exclusion': 'If we remove {Z1,Z0}→{v0}, then ¬({Z1,Z0}→y)'}},
 'frontdoor': None}

In [27]:
# dir(estimate)
estimate.value

10.87681181821216

In [24]:
# dir(estimate.estimator)
estimate.interpret()

Increasing the treatment variable(s) [v0] from 0 to 1 causes an increase of 10.87681181821216 in the expected value of the outcome [y], over the data distribution/population represented by the dataset.


In [38]:
dir(refute_results)
(refute_results.new_effect,refute_results.estimated_effect,refute_results.refutation_result,refute_results.refutation_type)

(10.650461820769264,
 10.87681181821216,
 {'p_value': 0.21999999999999997, 'is_statistically_significant': False},
 'Refute: Add a random common cause')

In [ ]:
dowhy.datasets